In [1]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.prompts import PromptTemplate

try:
    storage_context = StorageContext.from_defaults(persist_dir='../storage/cache/andrew/medical')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('../assets/AndrewHuberman/medical').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/medical/')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [3]:
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó."
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [4]:
from llama_index.postprocessor import SentenceTransformerRerank


rerank = SentenceTransformerRerank(
    model="sentence-transformers/all-mpnet-base-v2", 
    top_n=5
)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[rerank],
    text_qa_template = text_qa_template
)

In [16]:
now = time()
response = query_engine.query(
    "Nguyên nhân gây bệnh động mạch cảnh",
)
print(f"Elapsed: {round(time() - now, 2)}s")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 7.57s


In [17]:
print(response)

Nguyên nhân gây bệnh động mạch cảnh là do mảng xơ vữa làm dày thành mạch và làm hẹp dần lòng mạch.


In [18]:
for i in response.source_nodes:
    print('score: ', i.score)
    print('text: ', i.text)
    print('='*100)

score:  0.4993121
text:  Nếu quan sát sẽ thấy gương mặt của người bệnh thiếu biểu cảm sinh động, người bệnh cũng di chuyển chậm chạp hơn, tay chân ít cử động hơn,&#8230;
 <em>Các 
triệu chứng bệnh Parkinson
</em> thường bắt đầu ở một bên cơ thể và khi bệnh tiến triển, người bệnh có thể bị rối loạn chức năng vận động ở cả hai bên. Tuy nhiên, các triệu chứng thường chỉ nghiêm trọng ở một bên cơ thể (thường là bên có các dấu hiệu khởi phát bệnh).
<h2><b>Nguyên nhân bệnh Parkinson</b></h2> <em>Nguyên nhân bệnh Parkinson
</em> là do các tế bào thần kinh hạch nền (nhóm các nhân xám nằm sâu trong bán cầu đại não điều hòa vận động của con người) bị suy yếu và/hoặc mất. Thông thường, các tế bào thần kinh này sẽ tạo ra một chất hóa học được gọi là dopamine. Khi các tế bào thần kinh ở hạch nền chết hoặc bị suy yếu, chúng sẽ sản xuất ít dopamine hơn, giảm kích thích lên vỏ não và khiến người bệnh gặp khó khăn trong vận động.
 Tuy nhiên, tính đến thời điểm hiện tại, vẫn chưa tìm ra được nguyên nhân

In [ ]:
print(response.metadata)

# 2. Đánh giá RAG

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/duongtrongchi/project/RAG/rag-app/test/test_case.csv')

df.head()

In [ ]:
from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index.embeddings import SimilarityMode


evaluator = SemanticSimilarityEvaluator(
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)

list_res = []
list_result = []
passed = []
for i in range(0, len(df)):
    reference = df['References'][i]
    response = query_engine.query(df['Questions'][i])

    result = await evaluator.aevaluate(
        response=response.response,
        reference=reference,
    )

    list_res.append(response.response)
    list_result.append(result.score)
    passed.append(result.passing)


In [ ]:
df['Result'] = list_result
df['Passed'] = passed
df['Responses'] = list_res

In [ ]:
df.to_csv('../results.csv', index=False)

In [ ]:
response = query_engine.query(
    "Đối tượng nào dễ bị xuất huyết bao tử?",
)


print(response)

In [ ]:
print(response.source_nodes)

In [ ]:
# Embedding Similarity Evaluator

from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index.embeddings import SimilarityMode


evaluator = SemanticSimilarityEvaluator(
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)

reference = """
<h2><strong>Đối tượng nào dễ bị xuất huyết bao tử?</strong></h2> Xuất huyết dạ dày là tình trạng bệnh lý xảy ra phổ biến ở độ tuổi từ 20 &#8211; 50 tuổi, trong đó nam giới thường chiếm tỷ lệ cao hơn nữ giới bởi có xu hướng uống nhiều rượu bia hơn. Người lớn tuổi hay dùng các thuốc giảm đau chống viêm Non-steroid hoặc Aspirin cung có nguy cơ cao xuất huyết tiêu hóa.
"""

response = query_engine.query(
    "Bệnh bạch hầu là gì?",
)

result = await evaluator.aevaluate(
    response=response.response,
    reference=reference,
)

print("Score: ", result.score)
print("Passing: ", result.passing)